In [3]:
from tensorflow.keras import layers, losses, models as mds, optimizers
from collections import namedtuple
from random import sample, shuffle
from math import floor, ceil, sqrt, log, pi

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:

IMAGE_SIZE = 32

def sic_block(input_tensor, filters, strides, padding, activation):
    def variant_dims(size):
        n_variants = ceil(log(ceil(sqrt(size)), pi))
        return list(map(lambda x: 3**x, range(1, n_variants+1)))

    def make_layer(size):
        kwargs = dict(strides=strides, padding=padding)
        return layers.Conv2D(filters, size, **kwargs)

    size = min(input_tensor.shape[1:-1])
    variants = variant_dims(size)
    conv_layers = map(make_layer, variants)
    conv_outputs = list(map(lambda x: x(input_tensor), conv_layers))
    merged = tf.concat(conv_outputs, axis=-1)
    normalized = layers.BatchNormalization()(merged)
    output = layers.Activation(activation)(normalized)

    return output

def create_model():
    inp = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = sic_block(inp, filters=64, strides=1, padding='same', activation='relu')
    